In [ ]:
%pip install webdriver-manager
%pip install selenium

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
# function to grab all info from each property and store in dataframe

def get_farm_info():

    sleep(1)
    
    # get list of text elements
    text_container = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[4]/div/div/div[2]')
    property_name = text_container.find_element_by_xpath('./h2[@class="property-name"]').text
    #print(property_name)
    property_location = text_container.find_element_by_xpath('./p[1]').text
    #print(property_location)
    property_description = text_container.find_element_by_xpath('./p[3]').text
    #print(property_description)
    property_crops = text_container.find_element_by_xpath('./p[4]').text
    #print(property_crops)
    property_water = text_container.find_element_by_xpath('./p[5]').text
    #print(property_water)
    property_acquisition_date = text_container.find_element_by_xpath('./p[6]').text
    #print(property_acquisition_date)
    driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[4]/div/div/button').click()
    sleep(2)
    col_names = ['name', 'location', 'description', 'crops', 'water', 'acquisition']
    row_vals = [[property_name, property_location, property_description, property_crops, property_water, property_acquisition_date]]

    return(pd.DataFrame(row_vals, columns = col_names))


In [3]:
# full scraping code

url = 'https://www.gladstonefarms.com/'
# creates browser 
driver = webdriver.Chrome(ChromeDriverManager().install())

# navigates through company page and gets list of clickable elements for all the states
# 14 states that contain farms
driver.get(url)
driver.maximize_window()
sleep(2)
driver.find_element_by_xpath('//*[@id="mobile-nav"]/ul/li[2]/a').click()
sleep(1)
map = driver.find_element_by_xpath('//*[@id="g4864"]') 
sleep(1)
states = map.find_elements_by_xpath("*")
sleep(1)
#states = states[10:20]

true_states = []
for state in states:

    class_name = state.get_attribute("class")
    #print(class_name)
    if "existing" in class_name and class_name not in ["ca existing", "mi existing", "fl existing"]:
        true_states.append(state)

# i think becasue CA has so many properties it is causing problems if it is in the middle of the loop, so it is moved to the end of the loop
wa = driver.find_element_by_xpath('//*[@id="Washington"]')
true_states.append(wa)
ca = driver.find_element_by_xpath('//*[@id="California"]')
true_states.append(ca)


#true_states = [true_states[-1]]

all_data = pd.DataFrame()

for state in true_states:
    try:
        state.click()
        #print("success")
        sleep(1)
    except:
        print("No farm information")
        

    try:    
        states_container = driver.find_element_by_xpath('//*[@id="properties-by-state"]')
        sleep(1)
        properties_container = states_container.find_element_by_xpath('.//div[@class="state-property-list"]/div')
        #print('success')
        sleep(1)
    except:
        print("No Farm Information")

    properties = []
    try:
        properties = properties_container.find_elements_by_xpath('.//div[@class="result-row"]')
        #print('success')
    except:
        print("No farm information")
    
    # if type(properties) != "list":
    #     properties = [properties]

    state_df = pd.DataFrame()
    for prop in properties:
        try:
            # another for loop here
            prop.find_element_by_xpath('./a[@class="fancybox-property-modal arrow"]').click()
            current_prop = get_farm_info()
            state_df = pd.concat([state_df, current_prop])
        except:
            print("No Farm Information")
            
    sleep(1)
    all_data = pd.concat([all_data, state_df])
    #driver.find_element_by_xpath('//*[@id="mobile-nav"]/ul/li[2]/a').click()
    sleep(1)

######## get florida and michigan data ########

# florida data

# opens florida properties list
action = webdriver.ActionChains(driver)
fl = driver.find_element_by_xpath('//*[@id="Florida"]')
action.move_to_element_with_offset(fl, 10, 10)
action.click()
action.perform()

# gets list of property links
states_container = driver.find_element_by_xpath('//*[@id="properties-by-state"]')
sleep(1)
properties_container = states_container.find_element_by_xpath('.//div[@class="state-property-list"]/div')
sleep(1)
properties = []
properties = properties_container.find_elements_by_xpath('.//div[@class="result-row"]')

# loop through properties and get info
state_df = pd.DataFrame()
for prop in properties:
    prop.find_element_by_xpath('./a[@class="fancybox-property-modal arrow"]').click()
    current_prop = get_farm_info()
    state_df = pd.concat([state_df, current_prop])

all_data = pd.concat([all_data, state_df])

# michigan data

action = webdriver.ActionChains(driver)
fl = driver.find_element_by_xpath('//*[@id="Michigan"]')
action.move_to_element_with_offset(fl, 60, 50)
action.click()
action.perform()

# gets list of property links
states_container = driver.find_element_by_xpath('//*[@id="properties-by-state"]')
sleep(1)
properties_container = states_container.find_element_by_xpath('.//div[@class="state-property-list"]/div')
sleep(1)
properties = []
properties = properties_container.find_elements_by_xpath('.//div[@class="result-row"]')

# loop through properties and get info
state_df = pd.DataFrame()
for prop in properties:
    prop.find_element_by_xpath('./a[@class="fancybox-property-modal arrow"]').click()
    current_prop = get_farm_info()
    state_df = pd.concat([state_df, current_prop])

all_data = pd.concat([all_data, state_df])

driver.quit()
all_data2 = all_data




====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.80/chromedriver_mac64.zip
Driver has been saved in cache [/Users/aaronmorris/.wdm/drivers/chromedriver/mac64/98.0.4758.80]
/var/folders/6z/j_nl9g2s2q57q4c0dg1sm14r0000gn/T/ipykernel_19626/3088525569.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
/var/folders/6z/j_nl9g2s2q57q4c0dg1sm14r0000gn/T/ipykernel_19626/3088525569.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="mobile-nav"]/ul/li[2]/a').click()
/var/folders/6z/j_nl9g2s2q57q4c0dg1sm14r0000gn/T/ipykernel_19626/3088525569.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_el

In [4]:
########## Cleaning Script ##########

clean_dat = pd.DataFrame()

# 1. split columns to remove title and colon from each data point
description_ = all_data.description.str.split(":", expand = True)
description = description_[1]
crops_ = all_data.crops.str.split(":", expand = True)
crops = crops_[1]
water_ = all_data.water.str.split(":", expand = True)
water = water_[1]
acquisition_ = all_data.acquisition.str.split(":", expand = True)
acquisition = acquisition_[1]

clean_dat["name"] = all_data.name
clean_dat["location"] = all_data.location
clean_dat["description"] = description
clean_dat["crops"] = crops
clean_dat["water"] = water
clean_dat["acquisition"] = acquisition


# 2. get state from location 

clean_dat["state"] = clean_dat.location.str.split(",", expand = True)[1]
clean_dat = clean_dat.drop(columns = ["location"], axis = 1)

# 3. get acres and tillable acres from description

no_comma = clean_dat.description.str.replace(",", "")
split_description = no_comma.str.split("gross", expand = True)
clean_dat["total_acres"] = split_description[0]
clean_dat["tillable_acres"] = split_description[1].str.extract("(\d+)")
clean_dat = clean_dat.drop(columns = ["description"], axis = 1)







0    NaN
Name: tillable_acres, dtype: object